# Programmierübung 1: Lösung linearer Gleichungssysteme mittels LU-Zerlegung

### Vorbereitung
Bitte installieren Sie zunächst die benötigten Bibliotheken für diese Programmierübung. Wechseln Sie dazu zunächst in einem Terminal in das (entpackte) Verzeichnis dieser Programmierübung. Dort sollte sich neben diesem Notebook auch ein Verzeichnis "cer_pex1_lib" befinden.
Installieren Sie dieses mittels
```bash
pip install ./cer_pex1_lib
```
Bei der Installation dieses Pakets werden alle in dieser Übung benötigten Pakete automatisch mitinstalliert.

### Module importieren
Hier werden die benötigten Module für diese Programmierübung importiert.
Bitte importieren Sie keine weiteren Module. In dieser Programmierübung ist die Verwendung von Funktionen aus dem Sub-Modul numpy.linalg (insbesondere die Funktionen numpy.linalg.solve und numpy.linalg.inv) nicht erlaubt!

In [ ]:
import numpy as np
from typing import Tuple, Union

try:
    from cer_pex1 import test_np_equal, test_np_almost_equal, test_true
except ImportError:
    print(
        "I did not find the cer_pex1 library. Please make sure you installed it via \"pip install /path/to/cer_pex1_lib\".")
    raise

### Motivation

Die numerische Lösung großer linearer Gleichungssysteme tritt sehr häufig im Kern vieler Berechnungsalgorithmen der Mathematik, Informatik, Ingenieur-, Natur- und Wirtschftswissenschaften auf. Daher ist deren effiziente Lösung von großer Relevanz. Ein Beispiel wird im Laufe der Vorlesung die Inverskinematik eines Roboterarms sein, bei der wir die Gelenkstellungen des Arms für eine bestimmte Endposition bestimmen wollen. Solche Systeme sind im Allgemeinen nichtlinear, lassen sich aber um bestimmte Punkte (sog. *Ruhelagen*) linearisieren und dann als lineares Gleichungssystem lösen.

Betrachten wir das Gleichungssystem $Ax=b$, wobei $A\in\mathbb{R}^{n\times n}$ die quadratische Koeffizientenmatrix, $b\in\mathbb{R}^{n}$ der Vektor der Koeffizienten der rechten Seite, und $x\in\mathbb{R}^{n}$ der gesuchte Lösungsvektor ist. Existiert eine eindeutige Lösung für $x$, können wir diese direkt über $x=A^{-1}b$ erhalten. Im Falle von großen Gleichungssystemen ist eine exakte Bestimmung der Inversen jedoch ineffizient. Numerische Methoden zur direkten Berechnung der Inversen sind auch häufig instabil. 

In dieser Programmierübung implementieren wir deswegen die *LU-Zerlegung* der Matrix $A$, welche uns eine effizientere numerische Lösung großer linearer Gleichungssysteme ermöglicht. Die grundlegende Idee ist, die Matrix $A$ in ein Matrixprodukt $A=LU$ aufzuteilen, wobei $L$ eine untere Dreiecksmatrix, und $U$ eine obere Dreiecksmatrix ist. Sobald uns diese Zerlegung vorliegt, können wir das Gleichungssystem für verschiedene rechte Seiten $b$ in $\mathcal{O}(n^{2})$ lösen. Dieses Verfahren wird sich als numerisch stabil erweisen, insbesondere durch Implementierung einer Pivotsuche in Teilaufgabe 3.

Sofern nicht explizit anders angegeben nehmen wir an, dass die dem Gleichungssystem zugrunde liegende Matrix regulär ist, sodass eine eindeutige Lösung existiert und die Zerlegung möglich ist.

## Aufgabe 1: Lösen linearer Gleichungssysteme bei gegebener LU-Zerlegung

### 1.1 Vorwärtslösung
Betrachten wir ein gegebenes lineares Gleichungssystem $Ly=b$, wobei $L$ eine untere Dreiecksmatrix ist, also alle Einträge oberhalb der Hauptdiagonalen 0 sind: 
$$
L = \begin{bmatrix}
l_{11} & 0 & \cdots & 0 \\
l_{21} & l_{22} & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
l_{n1} & l_{n2} & \cdots & l_{nn}
\end{bmatrix}
$$
Die Lösung eines solchen Gleichungssystems ist einfach möglich: 

Schreiben wir zunächst die erste Zeile aus, so erhalten wir $l_{11}y_{1}=b_{1}$ und erhalten direkt $y_{1}$.

Wenn wir nun die zweite Zeile ausschreiben, erhalten wir $l_{21}y_{1}+l_{22}y_{2}=b_{2}$. Da wir $y_{1}$ im vorherigen Schritt bestimmt haben, ist nur $y_{2}$ unbekannt und lässt sich direkt aus der Gleichung bestimmen.

Dieses Vorgehen lässt sich iterativ Zeile für Zeile wiederholen. Insgesamt erhalten wir folgenden Algorithmus zur Berechnung des Lösungsvektors $y$:

$y_{1}=\frac{b_{1}}{l_{11}}$

$y_{i}=\frac{1}{l_{ii}}(b_{i}-\sum_{j=1}^{i-1}l_{ij}y_{j})$ FOR i=2..n

Implementieren Sie die Funktion `solve_forward`, die diesen Algorithmus anwendet, um $y$ zu bestimmen.

*Hinweis:* Beachten Sie, dass Indizes in Python / Numpy bei 0 beginnen und nicht bei 1.

In [ ]:
def solve_forward(L: np.array, b: np.array) -> np.array:
    """Solve a linear equation Ly=b, where L is a lower triangular matrix, using the presented algorithm.

    Args:
        L (np.array): Coefficient matrix with lower triangular structure, shape (n,n)
        b (np.array): Coefficient vector of right side, shape (n,)

    Returns:
        np.array: Solution vector y, shape (n,)
    """
    # Implement this...
    return np.nan*np.ones(L.shape[1])


### Öffentliche Tests

In [ ]:
test_np_almost_equal(1, solve_forward(np.eye(3), np.array([3, -2, 4])), np.array([3, -2, 4]))

test_np_almost_equal(2, solve_forward(np.array([[2, 0], [3, -1]]), np.array([6, 16])), np.array([3, -7]))

### 1.2 Rückwärtslösung
Wir können analog vorgehen um ein Gleichungssystem $Ux=y$ zu lösen, wobei $U$ eine obere Dreiecksmatrix ist, also alle Einträge unterhalb der Hauptdiagonalen 0 sind:
$$
U = \begin{bmatrix}
u_{11} & u_{12} & \cdots & u_{1n} \\
0 & u_{22} & \cdots & u_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & u_{nn}
\end{bmatrix}
$$

Auch hier können wir einen Algorithmus wie in Aufgabenteil 1.1 finden, der eine direkte Lösung für $x$ erlaubt. Implementieren Sie diesen analog zu Aufgabe 1.1 in der Funktion `solve_backward`.

*Hinweis:* Starten Sie in der letzten Zeile und arbeiten Sie sich nach oben.

In [ ]:
def solve_backward(U: np.array, y: np.array) -> np.array:
    """Solve a linear equation Ux=y, where U is a upper triangular matrix, using the presented algorithm.

    Args:
        U (np.array): Coefficient matrix with upper triangular structure, shape (n,n)
        y (np.array): Coefficient vector of right side, shape (n,)

    Returns:
        np.array: Solution vector x, shape (n,)
    """
    # Implement this...
    return np.nan*np.ones(U.shape[1])


### Öffentliche Tests

In [ ]:
test_np_almost_equal(1, solve_backward(np.eye(3), np.array([3, -2, 4])), np.array([3, -2, 4]))

test_np_almost_equal(2, solve_backward(np.array([[2, 3], [0, -1]]), np.array([-5, -3])), np.array([-7, 3]))


### 1.3 Lösung beliebiger Gleichungssysteme
Nehmen wir an, dass wir eine Zerlegung einer Matrix $A$ in eine untere und obere Dreiecksmatrix gegeben haben: $A=LU$. Eingesetzt in das ursprüngliche Gleichungssystem $Ax=b$ ergibt sich $LUx=b$. Mithilfe unserer Funktionen aus 1.1 und 1.2 können wir dieses Gleichungssystem nun in zwei Schritten direkt lösen:

1. Löse das Gleichungssystem $Ly=b$ zur Bestimmung des *Hilfslösungsvektors* $y\in\mathbb{R}^{n}$.
2. Löse das Gleichungssystem $Ux=y$.

Indem wir 2 in 1 einsetzen erhalten wir unser ursprüngliches Gleichungssystem, und die Aufteilung ist deswegen korrekt. Da $L$ und $U$ eine untere bzw. obere Dreiecksstruktur aufweisen, ist die Gesamtlösung also sehr effizient bestimmbar (zweimal $\mathcal{O}(n^{2})$ -> $\mathcal{O}(n^{2})$). Nutzen Sie die Funktionen aus den Aufgaben 1.1 und 1.2 um die Funktion `solve_system` zu implementieren, welche das Gesamtsystem $Ax=b$ wie beschrieben löst.

In [ ]:
def solve_system(L: np.array, U: np.array, b: np.array) -> np.array:
    """Solve the system of linear equations LUx=b, where L is a lower triangular matrix, U an upper triangular matrix.
       You may assume that the system is solvable.

    Args:
        L (np.array): Coefficient matrix with lower triangular structure, shape (n,n)
        U (np.array): Coefficient matrix with upper triangular structure, shape (n,n)
        b (np.array): Coefficient vector of right side, shape (n,)

    Returns:
        np.array: Solution vector x, shape (n,)
    """
    # Implement this...
    return np.nan*np.ones(L.shape[1])


### Öffentliche Tests

In [ ]:
test_np_almost_equal(1, solve_system(np.eye(3), np.eye(3), np.array([3, -2, 4])), np.array([3, -2, 4]))

test_np_almost_equal(2, solve_system(np.array([[2, 0], [3, -1]]), np.array([[2, 3], [0, -1]]), np.array([-10, -12])), np.array([-7, 3]))

## Aufgabe 2: LU-Zerlegung

Wir sehen, dass die Lösung eines linearen Gleichungssystems $Ax=b$ (ohne spezielle Struktur) effizient möglich ist, wenn wir $A$ in das Matrixprodukt einer unteren Dreiecksmatrix $L$ und einer oberen Dreiecksmatrix $U$ aufteilen können. Doch wie erhalten wir eine solche Aufteilung, falls sie existiert?

In dieser Aufgabe implementieren wir einen Algorithmus, der die LU-Zerlegung für eine Matrix $A\in\mathbb{R}^{n\times n}$ durchführt. Die Grundlage hierfür ist der bekannte Algorithmus zur Gauß-Elimination. Sie können sich dabei an folgendem Beispiel orientieren.

### Beispiel
Wir führen die Gauß-Elimination für die Matrix 
$A = \begin{bmatrix}
1 & -1 & 2 \\
3 & -2 & 1 \\
-1 & 0 & 2
\end{bmatrix}$ durch. In der ersten Iteration wollen wir die erste Spalte der Matrix so umformen, dass nur der Eintrag in der ersten Zeile ungleich 0 ist. Vergleichen wir die erste Spalte in Zeile 1 und 2, sehen wir dass der Eliminationsfaktor in unserem Fall $-3=-\frac{a_{21}}{a_{11}}$ sein muss, da $a_{21}-3a_{11}=0$. Wir ziehen also das Dreifache von Zeile 1 von Zeile 2 ab und erhalten
$\begin{bmatrix}
1 & -1 & 2 \\
0 & 1 & -5 \\
-1 & 0 & 2
\end{bmatrix}$.
Analog vergleichen wir Zeile 1 und 3 und sehen, dass wir einen Eliminationsfaktor von +1 brauchen, da $a_{31}+1a_{11}=0$. Wir erhalten damit
$\begin{bmatrix}
1 & -1 & 2 \\
0 & 1 & -5 \\
0 & -1 & 4
\end{bmatrix}$.

Nun müssen wir noch die zweite Spalte so umformen, dass in der letzten Zeile eine 0 steht. Vergleichen wir hierzu Zeile 2 und 3 ergibt sich ein Eliminationsfaktor von +1. Wir erhalten also
$U=\begin{bmatrix}
1 & -1 & 2 \\
0 & 1 & -5 \\
0 & 0 & -1
\end{bmatrix}$.

Die erhaltene Matrix ist die gesuchte Matrix $U$. Doch wie erhalten wir $L$? Es lässt sich zeigen, dass in der zugehörigen Matrix $L$ auf der Hauptdiagonalen nur Einsen stehen. Der Eintrag $l_{ij}$ **unterhalb** der Hauptdiagonalen ist der **negative** Eliminationsfaktor, den wir benutzt haben, um eine 0 an der entsprechenden Stelle in $U$ zu bekommen. Oberhalb der Hauptdiagonale stehen nur Nullen. In unserem Fall ergibt sich also 
$L=\begin{bmatrix}
1 & 0 & 0 \\
3 & 1 & 0 \\
-1 & -1 & 1
\end{bmatrix}$.

### 2.1 Implementierung der LU-Zerlegung
Implementieren Sie die LU-Zerlegung in der Funktion *decompose_lu*. Orientieren Sie sich am gezeigten Beispiel, also iterieren Sie über die Spalten und erzeugen nacheinander die obere Dreiecksstruktur. Die Funktion soll die Matrizen $L$ und $U$ zurückgeben.

In [ ]:
def decompose_lu(A: np.array) -> Tuple[np.array, np.array]:
    """Decompose the given matrix A into LU structure. You may assume that A is decomposable, i.e., 
       there are no 0 entries on the main diagonal.

    Args:
        A (np.array): Matrix that should be decomposed into LU, shape (n,n)

    Returns:
        Tuple[np.array, np.array]: L and U matrices, each in shape (n,n)
    """
    # Implement this...
    return np.nan*np.ones(A.shape), np.nan*np.ones(A.shape)


### Öffentliche Tests

In [ ]:
[L, U] = decompose_lu(np.array([[2, 3], [6, 8]])) 
test_np_almost_equal(1, L, np.array([[1, 0], [3, 1]]))
test_np_almost_equal(2, U, np.array([[2, 3], [0, -1]]))

### 2.2 Lösen des Gleichungssystems
Rufen Sie nun die Funktionen aus Aufgaben 1 und 2 auf, um das allgemeine Gleichungssystem $Ax=b$ mittels LU-Zerlegung zu lösen.

In [ ]:
def solve(A: np.array, b: np.array) -> np.array:
    """Solve the system of linear equations Ax=b using the implemented LU decomposition of A.
       You may assume that the system is solvable.

    Args:
        A (np.array): Coefficient matrix of linear equation, shape (n,n)
        b (np.array): Coefficient vector of right side, shape (n,)

    Returns:
        np.array: Solution vector x, shape (n,)
    """
    # Implement this...
    return np.nan*np.ones(A.shape[1])


### Öffentliche Tests

In [ ]:
x = solve(np.array([[2, 3], [6, 8]]), np.array([2, -3])) 
test_np_almost_equal(1, x, np.array([-12.5, 9]))

## Aufgabe 3: Pivotsuche

Bei der Bestimmung der Eliminationsfaktoren führen wir eine Division durch die Diagonalelemente $a_{ii}$ durch. Wir müssen hier zwei problematische Fälle betrachten:
1. Ist ein $a_{ii}$ gleich 0, so ist die versuchte Division nicht möglich und führt entweder zu einem Programmabbruch, oder `inf` Einträgen.
2. Ist ein $a_{ii}$ zwar nicht exakt 0, aber betragsmäßig sehr klein, führt die Division durch dieses Element zu hohen Eliminationsfaktoren und unsere Funktion wird damit numerisch instabil.
In dieser Aufgabe sichern wir die Korrektheit und erhöhen die numerische Stabilität der LU-Zerlegung durch geeignete Pivotsuche.

### Beispiele

Betrachten wir das Beispiel $A = \begin{bmatrix}
0 & -1\\
3 & -2 
\end{bmatrix}$. Bei der Bestimmung des Eliminationsfaktors ($\frac{a_{21}}{a_{11}}$) wie in Aufgabe 2 führen wir eine Division durch 0 durch, die zu einem Fehler führt:

In [ ]:
A = np.array([
    [0, -1],
    [3, -2]
])
try:
    decompose_lu(A)
except RuntimeWarning as w:
    print(w)

Und das obwohl die Matrix $A$ invertierbar ist! 

Wir stoßen auf ähnliche Probleme, wenn ein Pivotelement zwar nicht 0, aber nahe 0 ist. Dann teilen wir bei der Bestimmung des Eliminationsfaktors durch eine sehr kleine Zahl, welche zu Ungenauigkeiten aufgrund der Gleitpunktarithmetik des Computers und damit zu numerischer Instabilität führt.
Ein Beispiel, bei dem dieses Problem auftritt ist im letzten Teil dieser Programmierübung zu finden.

Offensichtlich müssen wir unseren Algorithmus für den Fall von Pivotelementen gleich oder betragsmäßig nahe 0 anpassen. Die grundlegende Idee ist hierbei, dass der Gauß-Algorithmus beliebige Zeilenvertauschungen erlaubt. Wir können also die betrachtete Zeile mit dem kritischen Pivotelement mit einer anderen Zeile unterhalb der betrachteten Zeile tauschen, welche in dieser Spalte ein Element ungleich 0 hat. Danach lässt sich der Algorithmus problemlos anwenden. Im betrachteten Beispiel tauschen wir also Zeile 1 und 2 und erhalten direkt die gewünschte Zerlegung.

Für eine korrekte Zerlegung müssen wir uns allerdings merken, welche Zeilen wir vertauscht haben. Dies können wir über eine Permutationsmatrix $P\in\mathbb{R}^{n\times n}$ darstellen. Vertauschen wir die Zeilen i und j, ist die zugehörige Permutationsmatrix die Einheitsmatrix, in welcher die Spalten (bzw. Zeilen) i und j vertauscht sind. Im Falle unseres Beispiels wäre also $P = \begin{bmatrix}
0 & 1\\
1 & 0 
\end{bmatrix}$.

Mittels dieses Algorithmus erhalten wir allgemein die Zerlegung $PA=LU$, wobei $L$ eine untere und $U$ eine obere Dreiecksmatrix ist. $P$ ist die resultierende Permutationsmatrix, welche das Matrixprodukt aller durchgeführten Permutationen $P=P_{n}\ldots P_{1}$, wobei $P_{i}$ die Permutationsmatrix der $i$-ten Iteration des Algorithmus ist, also die Permutation, die in der $i$-ten Spalte durchgeführt wurde.

### 3.1 Suche nach geeignetem Tauschelement
Zunächst suchen wir ein geeignetes Element, das wir als neues Pivotelement tauschen wollen. Wie oben beschrieben soll dieses Pivotelement das betragsmäßig größte Element sein, das in der aktuell betrachteten Spalte auf bzw. unter Hauptdiagonalen steht. Bei mehreren betragsmäßig gleich großen Elementen soll hier das erste ausgewählt werden. Implementieren Sie die Funktion `find_swap_row`, die diese Suche für die $j$-te Spalte der Matrix $A$ implementiert und den Index der entsprechenden Zeile zurückgibt. Falls alle Elemente $a_{jk}$ mit $k\geq j$ in der betrachteten Spalte Null sind, soll die Funktion eine -1 zurückgeben.

In [ ]:
def find_swap_row(A: np.array, j: int) -> int:
    """Find the row in column j of matrix A, which has the absolute maximal value (excluding elements that are above the main diagonal)

    Args:
        A (np.array): Matrix A, shape (n,n)
        j (int): Index of column in which we want to search.

    Returns:
        int: Index of found absolute maximum if this is unequal 0, else -1
    """
    # Implement this...
    return -1


### Öffentliche Tests

In [ ]:
A = np.array([[ 8, 10,  6,  2], [ 4,  7,  1,  2], [ 6,  9,  4,  5], [ 2,  3,  5,  1]])
test_true(1, find_swap_row(A, 0) == 0)
test_true(2, find_swap_row(A, 1) == 2)
test_true(3, find_swap_row(A, 2) == 3)
test_true(4, find_swap_row(A, 3) == 3)

### 3.2 Tauschen von Zeilen
Implementieren Sie die Funktion `swap_rows`, die zwei Zeilen in einem Numpy-Array $A$ vertauscht. Die Funktion soll zwei Matrizen zurückgeben: Die Matrix $A$, in welcher die entsprechenden Zeilen vertauscht wurden, und die zugehörige Permutationsmatrix $P$.

*Hinweis*: Der Fall $i=j$ soll berücksichtigt werden.

In [ ]:
def swap_rows(A: np.array, i: int, j: int) -> Tuple[np.array, np.array]:
    """Swap two rows at indices i and j in matrix A, and return the updated matrix A & the corresponding permutation matrix P.

    Args:
        A (np.array): Matrix in which to swap the rows, shape (n,n)
        i (int): Index of first row to swap
        j (int): Index of second row to swap

    Returns:
        Tuple[np.array, np.array]: Updated matrix A (where rows i and j are swapped) & the corresponding permutation matrix P
    """
    # Implement this...
    return np.nan*np.ones(A.shape[1]), np.nan*np.ones(A.shape[1])


### Öffentliche Tests

In [ ]:
A = np.array([[ 8, 10,  6,  2], [ 4,  7,  1,  2], [ 6,  9,  4,  5], [ 2,  3,  5,  1]])
[A2, P] = swap_rows(A.copy(), 0, 2)
test_np_equal(1, A2[[0]], A[[2]])
test_np_equal(2, A2[[2]], A[[0]])

### 3.3 LU-Zerlegung mit Pivotsuche
Nutzen Sie nun die Funktionen aus 3.1 und 3.2, um die LU-Zerlegung mit Pivotelementsuche in `decompose_lu_pivoted` zu implementieren. Die Funktion soll drei Matrizen zurückgeben: $L$ und $U$ wie in Aufgabe 2, und die resultierende Permutationsmatrix $P$. Im Falle, dass keine Zeilenvertauschung, sodass das Pivotelement ungleich 0 ist, möglich ist, soll die Funktion jeweils `None` für jede der drei Matrizen zurückgeben.

In [ ]:
def decompose_lu_pivoted(A: np.array) -> Union[Tuple[np.array, np.array, np.array], Tuple[None, None, None]]:
    """Decompose the given matrix A into LU structure using pivot search and return L, U, P. Return None for each matrix if not decomposable.

    Args:
        A (np.array): Matrix that should be decomposed into LU, shape (n,n)

    Returns:
        Union[Tuple[np.array, np.array, np.array], Tuple[None, None, None]]: Lower triang matrix L, Upper triang matrix U, permutation matrix P.
                                                                             If no decomposition possible, return None for each matrix.
    """
    # Implement this...
    return None, None, None


### 3.4 Lösen des Gleichungssystems
Lösen Sie nun das allgemeine Gleichungssystem $Ax=b$ mittels LU-Zerlegung und Pivotsuche. Berücksichtigen Sie hierbei den Einfluss der Permutationsmatrix $P$. Im Falle, dass das Gleichungssystem nicht lösbar ist, soll `None`zurückgegeben werden.

*Hinweis:* Überlegen Sie sich, welchen Einfluss die Matrix $P$ auf das Gleichungssystem hat.

In [ ]:
def solve_pivoted(A: np.array, b: np.array) -> Union[np.array, None]:
    """Solve the system of linear equations Ax=b using the implemented LU decomposition of A.
       Return None if system is not solvable.

    Args:
        A (np.array): Coefficient matrix of linear equation, shape (n,n)
        b (np.array): Coefficient vector of right side, shape (n,)

    Returns:
        np.array: Solution vector x, shape (n,)
    """
    # Implement this...
    return None


### Notwendigkeit der Pivotsuche in der LU-Zerlegung
Im Folgenden soll abschließend ein Fall betrachtet werden, der die Notwendigkeit der Pivotisierung veranschaulicht. Dazu wird das folgende Gleichungssystem $Kx=m$ definiert:

In [ ]:
K = np.array([[9, -7, 0], [-3, 2.333333333333, 6], [5, -1, 5]])
x = np.array([-1, -3, 2.5])
m = K@x

print('K=')
print(K)
print('\nx=')
print(x)
print('\nm=')
print(m)

Das Gleichungssystem $Kx=m$ ist gut konditioniert ($\operatorname{cond}(K)\approx 8.5$) und wirkt auf den ersten Blick unauffällig und einfach zu lösen. Die Anwendung von `solve` aus Aufgabenteil 2c) und die Berechnung des Residuums $K*x-m$ liefert jedoch das folgende Ergebnis:

In [ ]:
x_solve = solve(K, m)
error_ = np.abs(x-x_solve)
err_abs = np.sqrt(np.dot(error_, error_))
residuum = K@x_solve-m

print('Fehler:')
print(err_abs)
print('\nResiduum:')
print(residuum)

Das Ergebnis ist mit einem überraschend großen Fehler behaftet. Zum Vergleich lösen wir dasselbe Problem mit aktivierter Spaltenpivotsuche:

In [ ]:
x_solve = solve_pivoted(K, m)
error_ = np.abs(x-x_solve)
err_abs = np.sqrt(np.dot(error_, error_))
residuum = K@x_solve-m

print('Fehler:')
print(err_abs)
print('\nResiduum:')
print(residuum)

In diesem Fall ist der Fehler und das Residuum praktisch Null. Mit `solve_pivoted` kann man das Gleichungssystem also sehr viel genauer lösen als mit `solve`. Dies demonstriert, dass die Spaltenpivotisierung die Stabilität der Zerlegung erheblich erhöht.

Das gewählte Beispiel ist klein genug, um die einzelnen Schritte per Hand (oder computergestützt) nachrechnen zu können. Der letzte (unbepunktete) Teil dieser Aufgabe besteht darin, sich zu überlegen, an welcher Stelle bei `solve` das Problem auftritt, das zu dem großen Fehler führt. 
*Hinweis:* Verändern Sie die Anzahl der Nachkommastellen von $K_{11}=\frac{7}{3}$.